In [1]:
!pip install quandl
!pip install PyPortfolioOpt==1.5.5
!pip install arch


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install scikit-learn==1.2.2
import sklearn
import pypfopt
print(sklearn.__version__)
print(pypfopt.__version__)


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
1.2.2
1.5.5


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt
import datetime
import quandl
import math
import pypfopt
import random
from statsmodels.stats.diagnostic import het_arch
from arch import arch_model
from pypfopt import EfficientFrontier,objective_functions
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn import metrics
from copy import deepcopy

In [4]:
pypfopt.__version__

'1.5.5'

In [5]:
!pip install --upgrade yfinance
from pandas_datareader import data as pdr
import yfinance as yf


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [6]:
def modify_date(date):
    return date[:10]

In [7]:
def data_cleaning(data):
    if (data.isna()).any().any():
        missing = np.where(data.isna())
        rows=missing[0]
        cols=missing[1]
        print(rows)
        for row in range(len(rows)):
            col=cols[row]
            if row == len(data)-1:
                data.iloc[row,col]=data.iloc[row-1,col]**2/data.iloc[row-2,col]
            elif row ==0:
                data.iloc[row,col]=data.iloc[row+1,col]**2/data.iloc[row+2,col]
            else:
                data.iloc[row,col]=math.sqrt(data.iloc[row-1,col]*data.iloc[row+1,col])
        print(data)

In [8]:
def get_price_years_ago(data, date,time):
    year_ago=date-datetime.timedelta(days=int(time*365))
    price_year_ago=data[data.index<=year_ago]
    return price_year_ago

In [9]:
def max_drawdown_absolute(returns):
    r = returns.add(1).cumprod()
    dd = r.div(r.cummax()).sub(1)
    mdd = dd.min()
    end = dd.argmin()
    start = r.iloc[:end].argmax()
    return abs(mdd), start, end

In [10]:
def fit_garch(data,p,q):
    lag = max(p,q)
    length =res.shape[0]
    am = arch_model(res, vol='Garch', p=1, o=0, q=1, dist='Normal')
    split_date = one_year_monthly_return.index.values[lag-1]
    results = am.fit(last_obs=split_date)
    forecasts = results.forecast(horizon=1, start = length-lag, method='simulation', simulations=1000)
    sims = forecasts.variance
    std = np.sqrt(np.sum(sims)*12/lag)
    return std

In [11]:
def post_allocation(dfMVOweight):

    dfMVOweight['equal']=1/len(dfMVOweight)
    numofstocks=len(dfMVOweight)
    allocationweight=allocation.loc[numofstocks,:]
    dfMVOweight['proportion']=''
    dfMVOweight['adjusted proportion']=''
    dfMVOweight['direct']=''
    residual=0
    for i in range(numofstocks):
        dfMVOweight.iloc[i,4]=dfMVOweight.iloc[i,1]/np.sum(dfMVOweight['PR_SR'])
        if (dfMVOweight.iloc[i,4]+residual/(numofstocks-i))>allocationweight[0]:
            dfMVOweight.iloc[i,5]=allocationweight[0]
        else:
            dfMVOweight.iloc[i,5]=(dfMVOweight.iloc[i,4]+residual/(numofstocks-i))
        residual+=dfMVOweight.iloc[i,4]-dfMVOweight.iloc[i,5]
    for i in range(int(allocationweight[1])):
        dfMVOweight.iloc[i,6]=allocationweight[0]
    for i in range(int(allocationweight[1]),int(allocationweight[1]+allocationweight[3])):
        dfMVOweight.iloc[i,6]=allocationweight[2]
    return dfMVOweight

In [12]:
def output(weight):
    if stock_type == "S&P100":
      OEX=yf.download("^OEX", start=start_swan_date, end=end_swan_date)["Close"]

    elif stock_type == "S&P500":
      OEX=yf.download("SPY", start=start_swan_date, end=end_swan_date)["Close"]

    stock_price=yf.download(list(weight.index), start=start_swan_date, end=end_swan_date)["Close"]
    OEX.to_csv("oex.csv")
    injected_result = pd.DataFrame(index = stock_price.index )
    S1=np.array(10000*weight.T)/np.array(stock_price.iloc[0,:]).T
    portfolio_value = stock_price@S1.T
    injected_result["portfolio_value"]=portfolio_value
    S2 = 10000/float(OEX.iloc[0,0])
    injected_result["oex"]=S2*OEX
    return injected_result

# Step 1 - Preselection

In [13]:
def hist_var_cvar(series: pd.Series, alpha=0.05):
    x = series.dropna().values
    if x.size < 50:  # too few samples → estimates are unreliable
        return np.nan, np.nan
    x_sorted = np.sort(x)  # sort ascending so the left tail contains losses
    k = max(int(np.floor(alpha * x_sorted.size)), 1)
    var  = x_sorted[k-1]
    cvar = x_sorted[:k].mean()
    return var, cvar

def attach_cvar_feature(R: pd.DataFrame, alpha=0.05, colname="CVaR_5"):
    out = {}
    for c in R.columns:
        _, cvar = hist_var_cvar(R[c], alpha=alpha)
        out[c] = cvar
    return pd.Series(out, name=colname)

In [14]:
# Preselection parameters set up
method = "K_means_multiple"          # or avg or anyother method
stock_type = "S&P500"
end_swan = "Swan9"         #equals the last swan before the swan we are going to predict
start_swan="Swan5"
start = dt(2020,11,15)
end = dt(2022,1,1)
selected_stock_number = 22     #number of stock preselected
start_swan_date = dt(2022,1,3)
end_swan_date = dt(2025,10,13)
w1 = 0.17
w2 = 0.17
MA_days = 30
N = 7 # This parameter is set to help classification method like K-means

In [16]:
#read all stocks and their swan beta
if stock_type == "S&P100":
    portfolio_beta = pd.read_excel("All_SwanBeta100_MVO2.xlsx",index_col=0)
else:
    portfolio_beta = pd.read_excel("All_SwanBeta500.xlsx", sheet_name=1, index_col=0)


In [17]:
selected_swans_var=portfolio_beta.loc[:,start_swan:end_swan]
portfolio_beta =portfolio_beta[ selected_swans_var.T.isnull().sum()<selected_swans_var.shape[1]-1]


In [18]:
portfolio_beta

,Swan -3,Swan -2,Swan -1,Swan1,Swan2,Swan3,Swan4,Swan5,Swan6,Swan7,Swan8,Swan9,Swan10
Ticker,,,,,,,,,,,,,
A,NaN,NaN,NaN,NaN,NaN,NaN,2.129657,1.408493,2.084547,1.194656,0.682441,0.763786,0.786588
AAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.607495,1.783416,0.613406,1.207817,1.892777,1.317201
AAPL,NaN,1.977229,-0.376531,0.803017,1.957671,0.810491,1.374259,0.838225,0.265718,1.771183,1.901494,0.909584,0.966618
ABBV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.196566,0.593982,0.933795,-0.257703
ABC,NaN,NaN,NaN,NaN,NaN,1.799041,-0.098415,0.500850,0.644569,1.249513,1.141485,0.660946,-0.234940
...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,NaN,NaN,NaN,NaN,NaN,-0.250673,-0.054483,0.745233,0.626758,1.146095,0.138012,1.372255,0.880616
ZBH,NaN,NaN,NaN,NaN,NaN,NaN,-0.296565,1.240598,1.076392,0.777202,1.295412,1.468092,0.573753
ZBRA,NaN,NaN,NaN,NaN,NaN,1.096156,0.459355,1.041248,1.660871,2.720121,0.815003,0.860525,2.257289


In [148]:
corr=portfolio_beta.corr()
selected_corr=corr.loc["Swan1":end_swan,"Swan1":end_swan]
corr1={}
for i in range(selected_corr.shape[1]):
    for j in range(i+1,selected_corr.shape[0]):
        selected_corr.iloc[j,i]=np.nan

In [149]:
selected_corr

,Swan1,Swan2,Swan3,Swan4,Swan5,Swan6,Swan7,Swan8,Swan9
Swan1,1.0,0.220713,0.216300,0.079589,-0.008747,0.165929,0.071879,0.245460,0.068291
Swan2,NaN,1.000000,0.374644,0.209545,0.289873,0.258901,0.003171,0.192801,-0.004762
Swan3,NaN,NaN,1.000000,0.102535,0.190376,0.340516,0.240442,0.336471,0.073121
Swan4,NaN,NaN,NaN,1.000000,0.031385,0.170730,0.079042,0.177198,0.046191
Swan5,NaN,NaN,NaN,NaN,1.000000,0.487621,0.182212,0.256023,0.449046
Swan6,NaN,NaN,NaN,NaN,NaN,1.000000,0.281908,0.497957,0.398578
Swan7,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.255372,0.177822
Swan8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.294966
Swan9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000


In [89]:
avg=selected_corr.mean(axis=1)
weight_swan = avg/sum(avg)
selected_beata=portfolio_beta.loc[:,start_swan:end_swan].fillna(0)
#portfolio_beta["corr_beta"]=selected_beata@weight_swan
#portfolio_beta=portfolio_beta.sort_values(by=["corr_beta"])

In [150]:
portfolio_beta

,Swan -3,Swan -2,Swan -1,Swan1,Swan2,Swan3,Swan4,Swan5,Swan6,Swan7,Swan8,Swan9,Swan10
Ticker,,,,,,,,,,,,,
A,NaN,NaN,NaN,NaN,NaN,NaN,2.129657,1.408493,2.084547,1.194656,0.682441,0.763786,0.786588
AAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.607495,1.783416,0.613406,1.207817,1.892777,1.317201
AAPL,NaN,1.977229,-0.376531,0.803017,1.957671,0.810491,1.374259,0.838225,0.265718,1.771183,1.901494,0.909584,0.966618
ABBV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.196566,0.593982,0.933795,-0.257703
ABC,NaN,NaN,NaN,NaN,NaN,1.799041,-0.098415,0.500850,0.644569,1.249513,1.141485,0.660946,-0.234940
...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,NaN,NaN,NaN,NaN,NaN,-0.250673,-0.054483,0.745233,0.626758,1.146095,0.138012,1.372255,0.880616
ZBH,NaN,NaN,NaN,NaN,NaN,NaN,-0.296565,1.240598,1.076392,0.777202,1.295412,1.468092,0.573753
ZBRA,NaN,NaN,NaN,NaN,NaN,1.096156,0.459355,1.041248,1.660871,2.720121,0.815003,0.860525,2.257289


In [151]:
!pip install hmmlearn

In [152]:
from hmmlearn.hmm import GaussianHMM
from sklearn.cluster import KMeans
from sklearn import metrics
import numpy as np
import pandas as pd

In [153]:
def get_state_distribution(series, n_states=3):
    series = series.dropna()
    if len(series) < n_states + 2:
        return [1.0 / n_states] * n_states
    model = GaussianHMM(n_components=n_states, covariance_type="diag", n_iter=200, random_state=0)
    model.fit(series.values.reshape(-1, 1))
    hidden_states = model.predict(series.values.reshape(-1, 1))
    total = len(hidden_states)
    dist = [(hidden_states == i).sum() / total for i in range(n_states)]
    return dist


In [ ]:
# --- CVaR estimation window (reuse the same period as used in preselection)
cvar_start = start_swan_date
cvar_end   = end_swan_date

tickers_A = selected_beta.index.tolist()     # candidates before running K-means
px_A = yf.download(tickers_A, start=cvar_start, end=cvar_end, progress=False)["Close"]
if isinstance(px_A, pd.Series): px_A = px_A.to_frame()
R_A = px_A.pct_change().dropna(how="all")

cvar_A = attach_cvar_feature(R_A, alpha=0.05, colname="CVaR_5")
# remove excessively large tail losses (based on absolute value) or NaNs (soft mask)
thr = np.nanpercentile(cvar_A.abs(), 90)   # cut top 10% outliers
keep = cvar_A.abs() <= thr
selected_beta = selected_beta.loc[keep.index[keep]].copy()



/var/folders/fm/vxj_bbsj65n9s3s_fqtcvk5r0000gn/T/ipykernel_43704/3583682855.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  px_A = yf.download(tickers_A, start=cvar_start, end=cvar_end, progress=False)["Close"]


In [156]:
if method == "gamma":
    portfolio_beta["avg"] = portfolio_beta.loc[:, start_swan:end_swan].mean(axis=1)
    portfolio_beta["avg_sd"] = portfolio_beta.loc[:, start_swan:end_swan].mean(axis=1) + portfolio_beta.loc[:, start_swan:end_swan].std(axis=1, ddof=0)
    portfolio_beta = portfolio_beta.sort_values(by=["avg_sd"])
    preselected_stock = portfolio_beta.iloc[:selected_stock_number, :]
elif method == "average":
    portfolio_beta["avg"] = portfolio_beta.loc[:, start_swan:end_swan].mean(axis=1)
    portfolio_beta = portfolio_beta.sort_values(by=["avg"])
    preselected_stock = portfolio_beta.iloc[:selected_stock_number, :]

elif method == "K_means":
    selected_beta = portfolio_beta.loc[:, start_swan:end_swan].dropna()

    # === Inject HMM Features ===
    hmm_features = []
    for idx in selected_beta.index:
        series = selected_beta.loc[idx]
        dist = get_state_distribution(series, n_states=3)
        hmm_features.append(dist)
    hmm_df = pd.DataFrame(hmm_features, index=selected_beta.index,
                          columns=["HMM_state0", "HMM_state1", "HMM_state2"])
    selected_beta = pd.concat([selected_beta, hmm_df], axis=1)
    # ============================

    Y_predic = selected_beta[end_swan].copy()
    Kmean = KMeans(n_clusters=N, random_state=1)
    Kmean.fit(selected_beta)
    ct = Kmean.cluster_centers_
    labels = Kmean.labels_
    current_dbscore = metrics.davies_bouldin_score(selected_beta, labels)
    print(current_dbscore)
    selected_beta["Group"] = labels
    res = ct.mean(axis=1)
    res[res < 0] = 0
    min_group = np.where(res == min(res))
    for i in range(N):
        Y_predic[selected_beta["Group"] == i] = res[i]
    selected_beta["avg"] = Y_predic.values
    selected_beta = selected_beta.sort_values(by=["avg"])
    first_group = selected_beta[selected_beta["Group"] == selected_beta["Group"][0]]
    if first_group.shape[0] < 10:
        second_group = selected_beta[selected_beta["Group"] == selected_beta["Group"][first_group.shape[0]]]
        preselected_stock = pd.concat([first_group, second_group], axis=0, ignore_index=False)
    else:
        preselected_stock = first_group

elif method == "K_means_multiple":
    selected_beta = portfolio_beta.loc[:, start_swan:end_swan].dropna()

    # === Inject HMM Features ===
    hmm_features = []
    for idx in selected_beta.index:
        series = selected_beta.loc[idx]
        dist = get_state_distribution(series, n_states=3)
        hmm_features.append(dist)
    hmm_df = pd.DataFrame(hmm_features, index=selected_beta.index,
                          columns=["HMM_state0", "HMM_state1", "HMM_state2"])
    selected_beta = pd.concat([selected_beta, hmm_df], axis=1)
    # ============================

    Y_predic = selected_beta[end_swan].copy()
    seed_dict = {}
    for r in range(1, 1000):
        Kmean = KMeans(n_clusters=N, random_state=r, n_init=10)
        Kmean.fit(selected_beta)
        labels = Kmean.labels_
        dbscore = metrics.davies_bouldin_score(selected_beta, labels)
        seed_dict[r] = dbscore
    sorted_dict = sorted(seed_dict.items(), key=lambda x: x[1])

    stock_list = []
    max_length = 0
    for index in range(0, 5):
        selected_beta = portfolio_beta.loc[:, start_swan:end_swan].dropna()

        # === Inject HMM Features ===
        hmm_features = []
        for idx in selected_beta.index:
            series = selected_beta.loc[idx]
            dist = get_state_distribution(series, n_states=3)
            hmm_features.append(dist)
        hmm_df = pd.DataFrame(hmm_features, index=selected_beta.index,
                              columns=["HMM_state0", "HMM_state1", "HMM_state2"])
        selected_beta = pd.concat([selected_beta, hmm_df], axis=1)
        # ============================

        Kmean = KMeans(n_clusters=N, random_state=sorted_dict[index][0], n_init=10)
        Kmean.fit(selected_beta)
        ct = Kmean.cluster_centers_
        Kmean.labels_
        selected_beta["Group"] = Kmean.labels_
        res = ct.mean(axis=1)
        res[res < 0] = 0
        min_group = np.where(res == min(res))
        for i in range(N):
            Y_predic[selected_beta["Group"] == i] = res[i]
        selected_beta["avg"] = Y_predic.values
        selected_beta = selected_beta.sort_values(by=["avg"])
        first_group = selected_beta[selected_beta["Group"] == selected_beta["Group"][0]]
        if first_group.shape[0] < 10:
            second_group = selected_beta[selected_beta["Group"] == selected_beta["Group"][first_group.shape[0]]]
            preselected_stock = pd.concat([first_group, second_group], axis=0, ignore_index=False)
        else:
            preselected_stock = first_group
        if preselected_stock.shape[0] > max_length:
            max_length = preselected_stock.shape[0]
        stock_list.extend(preselected_stock.index.values)
    print(stock_list)

    values, counts = np.unique(stock_list, return_counts=True)
    count_dict = {}
    preselected_stock = []
    for t in range(0, len(values)):
        count_dict[values[t]] = counts[t]
    sorted_count = sorted(count_dict.items(), key=lambda x: x[1], reverse=True)
    print(sorted_count)

    d = {'stock': [], 'frequency': []}
    df = pd.DataFrame(data=d)
    for item in sorted_count:
        df.loc[len(df.index)] = [item[0], item[1]]
    df.to_csv('stock_frequency.csv')

    for p in range(0, min(22, len(sorted_count))):
        if sorted_count[p][1] >= 0:
            preselected_stock.append(sorted_count[p][0])

    preselected_stock = sorted(preselected_stock)
    selected_beta = selected_beta.loc[preselected_stock]
    selected_beta["avg"] = selected_beta.loc[:, start_swan:end_swan].mean(axis=1)
    preselected_stock = selected_beta
    print(preselected_stock)

Fitting a model with 14 free scalar parameters with only 5 data points will result in a degenerate solution.
Fitting a model with 14 free scalar parameters with only 5 data points will result in a degenerate solution.
Fitting a model with 14 free scalar parameters with only 5 data points will result in a degenerate solution.
Fitting a model with 14 free scalar parameters with only 5 data points will result in a degenerate solution.
Fitting a model with 14 free scalar parameters with only 5 data points will result in a degenerate solution.
Fitting a model with 14 free scalar parameters with only 5 data points will result in a degenerate solution.
Fitting a model with 14 free scalar parameters with only 5 data points will result in a degenerate solution.
Fitting a model with 14 free scalar parameters with only 5 data points will result in a degenerate solution.
Fitting a model with 14 free scalar parameters with only 5 data points will result in a degenerate solution.
Fitting a model wit

['SBAC', 'GIS', 'VZ', 'RSG', 'LLY', 'BF-B', 'FE', 'LNT', 'MCD', 'WEC', 'AZO', 'CME', 'AWK', 'WELL', 'MDLZ', 'MKC', 'KO', 'ATO', 'DUK', 'KDP', 'HRL', 'CMS', 'CLX', 'SRE', 'CINF', 'HSY', 'ICE', 'CHD', 'IFF', 'SO', 'CNP', 'CCI', 'SJM', 'CPB', 'SBUX', 'KMB', 'MKTX', 'EXC', 'EVRG', 'D', 'PPL', 'O', 'ORLY', 'AEP', 'AEE', 'DLTR', 'ED', 'PNW', 'YUM', 'PEAK', 'PEG', 'PEP', 'PG', 'DTE', 'XEL', 'WTW', 'CMG', 'ETR', 'WM', 'NI', 'AMT', 'NEM', 'NEE', 'PSA', 'MRK', 'ES', 'PEP', 'MKTX', 'CNP', 'BF-B', 'KDP', 'MNST', 'MCD', 'ED', 'WEC', 'WELL', 'AZO', 'CPB', 'AWK', 'GIS', 'CHD', 'RSG', 'FE', 'ATO', 'MKC', 'PEAK', 'CINF', 'ES', 'KO', 'PG', 'CCI', 'CL', 'ETR', 'EVRG', 'CLX', 'EXC', 'KMB', 'MDLZ', 'VZ', 'CME', 'CMG', 'CMS', 'O', 'MRK', 'PEG', 'NEE', 'NEM', 'NI', 'AEP', 'AEE', 'SO', 'SJM', 'HRL', 'HSY', 'YUM', 'ICE', 'DLTR', 'SRE', 'PNW', 'LLY', 'XEL', 'ORLY', 'PSA', 'WM', 'SBAC', 'AMT', 'WMT', 'WTW', 'SBUX', 'DUK', 'DTE', 'PPL', 'LNT', 'D', 'FE', 'LLY', 'D', 'LNT', 'MCD', 'SBUX', 'BF-B', 'WEC', 'MDLZ', 'S

In [ ]:
# Point in the pipeline right after K-means grouping, just before preselection
tickers_B = selected_beta.index.tolist()
px_B = yf.download(tickers_B, start=cvar_start, end=cvar_end, progress=False)["Close"]
if isinstance(px_B, pd.Series): px_B = px_B.to_frame()
R_B = px_B.pct_change().dropna(how="all")

cvar_B = attach_cvar_feature(R_B, alpha=0.05, colname="CVaR_5")
selected_beta = selected_beta.join(cvar_B, how="left")

# If PR (your 1/avg-style ratio) is missing, fall back to 'avg'
rank_base = "PR" if "PR" in selected_beta.columns else "avg"
lambda_cvar = 1.5  # CVaR weight (tune in the 1–2 range)

selected_beta["TailScore"] = (
    selected_beta[rank_base].rank(ascending=True) +
    lambda_cvar * selected_beta["CVaR_5"].abs().rank(ascending=True)
)
# Keep top N names with the smallest TailScore
N = selected_stock_number
preselected_stock = selected_beta.sort_values("TailScore").head(N)
# preselected_stock.to_csv("preselected_with_cvar.csv")


/var/folders/fm/vxj_bbsj65n9s3s_fqtcvk5r0000gn/T/ipykernel_43704/3163515922.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  px_B = yf.download(tickers_B, start=cvar_start, end=cvar_end, progress=False)["Close"]


In [158]:
ct

array([[ 0.66937012,  0.24302553,  0.34972701, -0.03029718,  0.88866834,
         0.34375   ,  0.2875    ,  0.36875   ],
       [ 0.69636269,  0.57045373,  1.03026465,  0.88608585,  0.79551634,
         0.38461538,  0.25641026,  0.35897436],
       [ 1.26988762,  2.06350426,  0.26016323,  1.13339128,  1.24660218,
         0.40571429,  0.25142857,  0.34285714],
       [ 0.77136433,  0.87796015,  0.13788672,  1.28509392,  0.92177137,
         0.36595745,  0.25531915,  0.3787234 ],
       [ 1.27619034,  1.40908894,  1.25355689,  1.15249243,  1.22223484,
         0.38058252,  0.25825243,  0.36116505],
       [ 1.10340121,  1.02053814,  0.65165517,  0.55837589,  1.32911315,
         0.36923077,  0.27384615,  0.35692308],
       [ 1.46831956,  2.16691257,  1.56054281,  1.72992688,  1.59133832,
         0.37674419,  0.27906977,  0.34418605]])

In [159]:
res

array([0.39006173, 0.62233541, 0.87169357, 0.62425956, 0.91419543,
       0.70788545, 1.18963002])

In [160]:
preselected_stock.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22 entries, ED to CCI
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Swan5       22 non-null     float64
 1   Swan6       22 non-null     float64
 2   Swan7       22 non-null     float64
 3   Swan8       22 non-null     float64
 4   Swan9       22 non-null     float64
 5   HMM_state0  22 non-null     float64
 6   HMM_state1  22 non-null     float64
 7   HMM_state2  22 non-null     float64
 8   Group       22 non-null     int32  
 9   avg         22 non-null     float64
 10  CVaR_5      22 non-null     float64
 11  TailScore   22 non-null     float64
dtypes: float64(11), int32(1)
memory usage: 2.1+ KB


In [161]:
#portfolio_beta["avg"]=portfolio_beta.loc[:,start_swan:end_swan]*weight

In [162]:
#order = list(preselected_stock.index)
#random.shuffle(order)
#preselected_stock = preselected_stock.reindex(index = order)
preselected_stock

,Swan5,Swan6,Swan7,Swan8,Swan9,HMM_state0,HMM_state1,HMM_state2,Group,avg,CVaR_5,TailScore
Ticker,,,,,,,,,,,,
ED,0.522844,-0.369009,-0.192322,0.007207,0.901315,0.2,0.4,0.4,0,0.174007,-0.027370,7.5
DUK,0.753130,-0.306652,0.261514,-0.370332,1.089892,0.4,0.2,0.4,0,0.285510,-0.026568,7.5
CMS,0.323398,0.149975,0.087549,0.016946,0.864892,0.2,0.4,0.4,0,0.288552,-0.028345,13.0
ATO,0.548667,0.468100,-0.135667,0.164406,0.959928,0.4,0.2,0.4,0,0.401087,-0.026656,19.0
CHD,0.597611,-0.086320,0.238839,-0.333092,0.583540,0.6,0.0,0.4,0,0.200116,-0.031716,20.5
AEE,1.139264,-0.008179,-0.139506,0.001823,0.833510,0.2,0.4,0.4,0,0.365382,-0.029160,22.5
AZO,-0.451106,-0.302806,-0.147543,-0.403255,0.955177,0.2,0.4,0.4,0,-0.069906,-0.034457,23.5
EVRG,0.762815,0.150107,0.011664,-0.139129,1.150501,0.4,0.2,0.4,0,0.387192,-0.029149,24.0
AEP,0.764573,0.012209,0.305262,-0.296072,0.959958,0.6,0.0,0.4,0,0.349186,-0.029576,24.5


In [163]:
preselected_stock["PR"]=1-preselected_stock["avg"]
preselected_stock.to_csv('preselected_stocks.csv')
preselected_stock

,Swan5,Swan6,Swan7,Swan8,Swan9,HMM_state0,HMM_state1,HMM_state2,Group,avg,CVaR_5,TailScore,PR
Ticker,,,,,,,,,,,,,
ED,0.522844,-0.369009,-0.192322,0.007207,0.901315,0.2,0.4,0.4,0,0.174007,-0.027370,7.5,0.825993
DUK,0.753130,-0.306652,0.261514,-0.370332,1.089892,0.4,0.2,0.4,0,0.285510,-0.026568,7.5,0.714490
CMS,0.323398,0.149975,0.087549,0.016946,0.864892,0.2,0.4,0.4,0,0.288552,-0.028345,13.0,0.711448
ATO,0.548667,0.468100,-0.135667,0.164406,0.959928,0.4,0.2,0.4,0,0.401087,-0.026656,19.0,0.598913
CHD,0.597611,-0.086320,0.238839,-0.333092,0.583540,0.6,0.0,0.4,0,0.200116,-0.031716,20.5,0.799884
AEE,1.139264,-0.008179,-0.139506,0.001823,0.833510,0.2,0.4,0.4,0,0.365382,-0.029160,22.5,0.634618
AZO,-0.451106,-0.302806,-0.147543,-0.403255,0.955177,0.2,0.4,0.4,0,-0.069906,-0.034457,23.5,1.069906
EVRG,0.762815,0.150107,0.011664,-0.139129,1.150501,0.4,0.2,0.4,0,0.387192,-0.029149,24.0,0.612808
AEP,0.764573,0.012209,0.305262,-0.296072,0.959958,0.6,0.0,0.4,0,0.349186,-0.029576,24.5,0.650814


# Step 2 - Download data and adjuste price based on Swan Beta

In [28]:
#next line is used to input preselected stock from excel, if you are going to preselect by your self, use this blocks and note next one.
#selected_stock_list =  pd.read_excel("19_stickers.xlsx",header = None).values[:,0]
#swan_beta=pd.read_excel("19_stickers.xlsx",header = None,index_col=0).iloc[:,0]

In [164]:
#Set the paramter to download tradeinfo from Yahoo Finance
selected_stock=preselected_stock
selected_stock_list=selected_stock.index.values
swan_beta=preselected_stock.loc[selected_stock_list,"avg"]
print(selected_stock_list)

['ED' 'DUK' 'CMS' 'ATO' 'CHD' 'AEE' 'AZO' 'EVRG' 'AEP' 'AWK' 'DTE' 'FE'
 'D' 'DLTR' 'CNP' 'CLX' 'EXC' 'CPB' 'ES' 'AMT' 'ETR' 'CCI']


In [165]:
#download data from yahoo finance
yahooDataFrame= yf.download(list(selected_stock_list), start=start, end=end)["Close"]


/var/folders/fm/vxj_bbsj65n9s3s_fqtcvk5r0000gn/T/ipykernel_43704/3756144450.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  yahooDataFrame= yf.download(list(selected_stock_list), start=start, end=end)["Close"]
[*********************100%***********************]  22 of 22 completed


In [166]:
yahooDataFrame

Ticker,AEE,AEP,AMT,ATO,AWK,AZO,CCI,CHD,CLX,CMS,...,D,DLTR,DTE,DUK,ED,ES,ETR,EVRG,EXC,FE
Date,,,,,,,,,,,,,,,,,,,,,
2020-11-16,72.869331,74.870682,207.298889,89.246780,151.050629,1146.020020,131.651596,83.194855,179.190460,57.098576,...,69.317764,95.610001,97.620461,78.780296,69.064674,78.323212,47.202290,47.149773,26.122753,23.407423
2020-11-17,69.646248,72.789307,207.376923,88.162941,145.338928,1124.209961,132.383759,82.476112,180.424362,55.171082,...,67.787552,93.820000,95.916451,77.047310,66.793266,76.100319,46.863255,46.282532,25.960011,23.604540
2020-11-18,68.315529,70.607208,206.093689,84.743942,142.510452,1131.650024,135.813950,81.331764,176.869370,53.157158,...,65.403679,93.650002,94.010246,75.894753,65.478477,74.441467,46.122379,45.006218,25.736998,23.448486
2020-11-19,67.702034,69.197243,206.726639,83.801048,142.930130,1133.760010,133.776535,81.520920,174.703598,52.361965,...,64.235909,94.550003,93.620354,76.035698,64.981209,73.147568,45.611721,44.357662,25.526041,23.850933
2020-11-20,67.788429,69.230827,201.723862,84.303337,143.897293,1114.199951,132.885193,81.293945,175.204041,52.941074,...,63.341949,94.970001,93.764755,76.765396,65.225647,73.512505,45.615906,45.035137,25.236723,22.996769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,78.200584,75.706734,254.070282,94.765114,169.912643,2047.060059,169.293091,96.036301,151.124084,56.842403,...,65.046989,138.470001,104.064629,89.040985,73.500488,76.639557,48.153835,57.732292,34.165234,34.673176
2021-12-28,78.977676,76.689140,256.497589,94.191399,172.282028,2059.939941,170.574661,95.864067,152.043793,57.402729,...,65.450249,139.289993,104.841690,90.144409,74.088638,77.361092,48.614388,58.468056,34.663914,35.136055
2021-12-29,79.415344,77.010826,256.844299,95.466354,173.438995,2104.040039,171.401505,96.629608,151.557434,57.980846,...,65.878670,140.300003,105.415649,90.308189,74.703087,77.833565,48.875084,58.613499,35.312172,35.298916


In [167]:
yahooDataFrame.to_csv('yahooDataFrame.csv')
yahooDataFrame=yahooDataFrame[selected_stock_list]
yahooDataFrame

Ticker,ED,DUK,CMS,ATO,CHD,AEE,AZO,EVRG,AEP,AWK,...,D,DLTR,CNP,CLX,EXC,CPB,ES,AMT,ETR,CCI
Date,,,,,,,,,,,,,,,,,,,,,
2020-11-16,69.064674,78.780296,57.098576,89.246780,83.194855,72.869331,1146.020020,47.149773,74.870682,151.050629,...,69.317764,95.610001,22.039562,179.190460,26.122753,42.044308,78.323212,207.298889,47.202290,131.651596
2020-11-17,66.793266,77.047310,55.171082,88.162941,82.476112,69.646248,1124.209961,46.282532,72.789307,145.338928,...,67.787552,93.820000,22.109840,180.424362,25.960011,41.606167,76.100319,207.376923,46.863255,132.383759
2020-11-18,65.478477,75.894753,53.157158,84.743942,81.331764,68.315529,1131.650024,45.006218,70.607208,142.510452,...,65.403679,93.650002,20.987560,176.869370,25.736998,41.319702,74.441467,206.093689,46.122379,135.813950
2020-11-19,64.981209,76.035698,52.361965,83.801048,81.520920,67.702034,1133.760010,44.357662,69.197243,142.930130,...,64.235909,94.550003,20.704775,174.703598,25.526041,41.555618,73.147568,206.726639,45.611721,133.776535
2020-11-20,65.225647,76.765396,52.941074,84.303337,81.293945,67.788429,1114.199951,45.035137,69.230827,143.897293,...,63.341949,94.970001,20.731281,175.204041,25.236723,41.269142,73.512505,201.723862,45.615906,132.885193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,73.500488,89.040985,56.842403,94.765114,96.036301,78.200584,2047.060059,57.732292,75.706734,169.912643,...,65.046989,138.470001,25.021553,151.124084,34.165234,36.697327,76.639557,254.070282,48.153835,169.293091
2021-12-28,74.088638,90.144409,57.402729,94.191399,95.864067,78.977676,2059.939941,58.468056,76.689140,172.282028,...,65.450249,139.289993,25.266510,152.043793,34.663914,37.723946,77.361092,256.497589,48.614388,170.574661
2021-12-29,74.703087,90.308189,57.980846,95.466354,96.629608,79.415344,2104.040039,58.613499,77.010826,173.438995,...,65.878670,140.300003,25.357231,151.557434,35.312172,37.602150,77.833565,256.844299,48.875084,171.401505


In [170]:
# === Simplified Market Drop Construction (use only Actual DD) ===
dd = 0.272  # 27.20%

# create a flat drop representation (constant or single-step)
market_drop = pd.DataFrame([dd], columns=["Swan Drop SPY"])

# save if needed
market_drop.to_csv("market_drop.csv", index=False)

market_drop


,Swan Drop SPY
0,0.272


In [171]:
swan_drop_pct=np.array(swan_beta)*np.array(market_drop)
yahooDataFrame=pd.DataFrame((1-swan_drop_pct)*np.array(yahooDataFrame),index=yahooDataFrame.index,columns=yahooDataFrame.columns)

In [172]:
yahooDataFrame=yahooDataFrame.dropna(how="all",axis = 1)
selected_stock_list=yahooDataFrame.columns.values
selected_stock_number = yahooDataFrame.shape[1]
selected_stock_list

array(['ED', 'DUK', 'CMS', 'ATO', 'CHD', 'AEE', 'AZO', 'EVRG', 'AEP',
       'AWK', 'DTE', 'FE', 'D', 'DLTR', 'CNP', 'CLX', 'EXC', 'CPB', 'ES',
       'AMT', 'ETR', 'CCI'], dtype=object)

In [173]:
yahooDataFrame.to_csv('yahooDataFrame.csv')
yahooDataFrame

Ticker,ED,DUK,CMS,ATO,CHD,AEE,AZO,EVRG,AEP,AWK,...,D,DLTR,CNP,CLX,EXC,CPB,ES,AMT,ETR,CCI
Date,,,,,,,,,,,,,,,,,,,,,
2020-11-16,65.795848,72.662311,52.617129,79.510353,78.666439,65.627284,1167.811042,42.184141,67.759571,141.190986,...,63.875448,92.449688,18.524057,165.077498,22.217755,37.757457,70.529195,187.610127,39.853667,112.671974
2020-11-17,63.631945,71.063906,50.840916,78.544756,77.986819,62.724524,1145.586276,41.408235,65.875882,135.852109,...,62.465377,90.718854,18.583125,166.214218,22.079341,37.363989,68.527504,187.680749,39.567414,113.298584
2020-11-18,62.379385,70.000855,48.985058,75.498755,76.904759,61.526057,1153.167809,40.266338,63.901036,133.208259,...,60.268668,90.554475,17.639858,162.939215,21.889665,37.106732,67.033726,186.519393,38.941881,116.234260
2020-11-19,61.905652,70.130855,48.252276,74.658726,77.083619,60.973534,1155.317914,39.686085,62.624987,133.600542,...,59.192582,91.424728,17.402180,160.944019,21.710244,37.318594,65.868584,187.092227,38.510723,114.490570
2020-11-20,62.138521,70.803886,48.785934,75.106217,76.868999,61.051343,1135.385931,40.292211,62.655382,134.504575,...,58.368810,91.830843,17.424458,161.405047,21.464175,37.061327,66.197206,182.564602,38.514257,113.727729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,70.021715,82.126167,52.381062,84.426661,90.808906,70.428696,2085.983927,51.652151,68.516216,158.821806,...,59.939983,133.892985,21.030394,139.221617,29.057994,32.955656,69.013082,229.939282,40.657072,144.886863
2021-12-28,70.582028,83.143900,52.897410,83.915535,90.646048,71.128557,2099.108714,52.310427,69.405315,161.036532,...,60.311581,134.685873,21.236278,140.068890,29.482128,33.877600,69.662816,232.136049,41.045924,145.983675
2021-12-29,71.167394,83.294961,53.430153,85.051399,91.369919,71.522728,2144.047354,52.440552,69.696448,162.117979,...,60.706366,135.662498,21.312528,139.620837,30.033481,33.768223,70.088273,232.449830,41.266034,146.691316


# Step3 - MVO

In [174]:
#Set the paramter for MVO process
Portfolio_Date=end

In [175]:
#data cleaning - filling missing data in yahooDataFrame
data_cleaning(yahooDataFrame)

In [51]:
#calculated rf
#bond_yield = quandl.get('FRED/DGS3MO',api_key='X4EQzCpMfeaTCTDthRH_')
#bond_yield = quandl.get('FRED/DGS3MO')
#bond_yield=np.log(bond_yield/100+1)
#date=list(map(str,bond_yield.index.values))
#date=list(map(modify_date,date))
#week=pd.to_datetime(date).strftime("%W").values
#week_1=np.roll(week,1)
#week_difference=np.where(week!=week_1)
#week_difference=np.delete(week_difference,0,None)
#rf_week=bond_yield.iloc[week_difference-1]

In [176]:
# Calculated R for 1 year and 0.5 year
today_price = yahooDataFrame.iloc[-MA_days:,:]
one_year_ago_data = get_price_years_ago(yahooDataFrame,end,1)
one_year_ago_date = one_year_ago_data.index.values[-1]
half_year_ago_data = get_price_years_ago(yahooDataFrame,end,0.5)
half_year_ago_date = half_year_ago_data.index.values[-1]
one_year_ago_price=yahooDataFrame.loc[one_year_ago_date:,:]
one_year_ago_price=yahooDataFrame.iloc[:MA_days,:]
half_year_ago_price=yahooDataFrame.loc[half_year_ago_date:,:]
half_year_ago_price=yahooDataFrame.iloc[:MA_days,:]

R_mv_one_year=np.mean(pd.DataFrame(np.array(today_price)/np.array(one_year_ago_price)-1,index=today_price.index,columns=today_price.columns),axis=0)
R_mv_half_year=np.mean(pd.DataFrame(np.array(today_price)/np.array(half_year_ago_price)-1,index=today_price.index,columns=today_price.columns),axis=0)



In [177]:
# Calculate MA
MA_Data=yahooDataFrame.rolling(MA_days).mean().dropna(axis=0)
MA_Data=MA_Data.loc[one_year_ago_date:,:]
MA_now = MA_Data.iloc[-1,:]
MA_one_year_ago = MA_Data.iloc[0,:]
MA_half_year_ago = MA_Data.loc[half_year_ago_date,:]

In [178]:
# Calculate daily return
daily_return = yahooDataFrame.pct_change().dropna()
log_daily_return = np.log(1+daily_return)

# Get week end stock price data
stock_date = yahooDataFrame.index.values
week=pd.to_datetime(stock_date).strftime("%W").values
week_1=np.roll(week,1)
week_difference=np.where(week!=week_1)
week_difference=np.delete(week_difference,0,None)
# Calculate weekly return

weekly_return=yahooDataFrame.iloc[week_difference-1].pct_change().dropna()
log_weekly_return = np.log(1+weekly_return)

# Get month end stock price data
stock_date = yahooDataFrame.index.values
month=pd.to_datetime(stock_date).strftime("%m").values
month_1=np.roll(month,1)
month_difference=np.where(month!=month_1)
month_difference=np.delete(month_difference,0,None)
# Calculate monthly return
monthly_return=yahooDataFrame.iloc[month_difference-1].pct_change().dropna()
log_monthly_return = np.log(1+monthly_return)

# Get semi-month end stock price data
semi_month = []
for i in range(1,len(month_difference)):
    semi_month.append((month_difference[i]-month_difference[i-1])//2+month_difference[i-1])
    semi_month.append(month_difference[i]-1)
# Calculate semi-monthly return
semi_monthly_return=yahooDataFrame.iloc[semi_month].pct_change().dropna()
log_semi_monthly_return = np.log(1+semi_monthly_return)

In [179]:
# Calculated Max Dropdown for each stock
stock_return_one_year=daily_return.loc[one_year_ago_date:,:]
mdd=[]
for column in stock_return_one_year.columns:
    mdd .append(max_drawdown_absolute(stock_return_one_year.loc[:,column]))
mdd=pd.DataFrame(mdd,index = stock_return_one_year.columns,columns=["MAXDD","start_index","end_index"])
mar=R_mv_one_year/mdd.iloc[:,0]

In [180]:
#calculate weekly sd for each stock
one_year_weekly_return=log_weekly_return.iloc[log_weekly_return.index >= one_year_ago_date]
weekly_std= np.std(one_year_weekly_return)*np.sqrt(one_year_weekly_return.shape[0])

#Calculate garch_std sd for each stock
one_year_monthly_return=log_monthly_return.iloc[log_monthly_return.index >= one_year_ago_date]
res_all = one_year_monthly_return - np.mean(one_year_monthly_return)
judge = one_year_monthly_return.shape[0]
for stock in one_year_monthly_return.columns:
    res=res_all[stock]
    h= het_arch(res)
    if judge < 100 or h ==0:
        garch_std = weekly_std

    else:#build Garch Model
        std = []
        if judge <=200:
            p = 0
            q = 6
        else:
            p = 1
            q = 1

        std.append(fit_garch(one_year_monthly_return[stock],p,q))
        garch_std = pd.DataFrame(std,index = one_year_monthly_return.columns)

#Calculate monthly sd for each stock
one_year_monthly_return=log_monthly_return.iloc[log_monthly_return.index >= one_year_ago_date]
monthly_std = np.std(one_year_monthly_return)*np.sqrt(12);

#Calculate semi_monthly sd for each stock
one_year_semi_monthly_return=log_semi_monthly_return.iloc[log_semi_monthly_return.index >= one_year_ago_date]
semi_monthly_std = np.std(one_year_semi_monthly_return)*np.sqrt(24);


In [181]:
# Calculated MaxDrop
DD_range = 90
total_range = 90
DD_end = yahooDataFrame.iloc[yahooDataFrame.index <= one_year_ago_date]
DD_start = yahooDataFrame.iloc[yahooDataFrame.index <= end-datetime.timedelta(total_range)]
max_DD=[-1]*selected_stock_number


if (DD_start.shape[0] != 1 ) and (DD_end.shape[0] != 1):
    DD_start = DD_start.index[-1]
    DD_end = DD_end.index[-1]
    while DD_start > DD_end :
        DD_start = yahooDataFrame.iloc[yahooDataFrame.index <= end-datetime.timedelta(total_range)]
        DD_mid = yahooDataFrame.iloc[yahooDataFrame.index <= end-datetime.timedelta(DD_range+total_range)]
        if (DD_start.shape[0] != 1 ) and (DD_mid.shape[0] != 1):
            DD_start = DD_start.index[-1]
            DD_mid = DD_mid.index[-1]
            price = yahooDataFrame.loc[DD_start:DD_mid,:]
            for i in range(len(yahooDataFrame.columns)):
                max_DD[i]=max(max_DD[i],max_drawdown_absolute(stock_return_one_year.iloc[:,i])[0])
            total_range = total_range + 1
        else:
            break

In [184]:
# Read OPT-data
optdata = pd.read_csv("optdata_s10d.csv")

# 인덱스: ticker
optdata["ticker"] = optdata["ticker"].astype(str).str.strip().str.upper().str.replace("/", "-", regex=False)
optdata = optdata.set_index("ticker")

# 기존 변수 세팅
P_current = MA_now
P_fwd = optdata["P_fwd"].loc[P_current.index]

R_fwd = P_fwd / P_current - 1
R_60 = P_current / MA_one_year_ago - 1
R_60_half = P_current / MA_half_year_ago - 1
R_MA_1ago = R_mv_one_year
R_MA_halfago = R_mv_half_year

# 빈 Series 생성
Rfwd_adjusted = pd.Series(index=R_MA_1ago.index, dtype=float)

# 조건 적용
for i in R_fwd.index:
    if R_fwd[i] < 0.5 * R_60[i]:
        Rfwd_adjusted[i] = R_60_half[i]
    elif R_fwd[i] > 1.3 * R_60[i]:
        if 1.3 * R_60[i] > 0.7 * R_fwd[i]:
            Rfwd_adjusted[i] = 1.3 * R_60[i]
        else:
            Rfwd_adjusted[i] = 0.7 * R_fwd[i]
    else:
        Rfwd_adjusted[i] = R_fwd[i]

# 음수 제거
Rfwd_adjusted[Rfwd_adjusted < 0] = 0


In [185]:
price_now = today_price.iloc[-1,:]
optimizer = optdata.loc[P_current.index,:]
FFC = pd.DataFrame([R_MA_1ago.values,R_MA_halfago.values,MA_now.values,price_now.values,semi_monthly_std.values,weekly_std.values,MA_one_year_ago.values,MA_half_year_ago.values,max_DD,R_60.values,R_60_half.values,P_fwd.values,Rfwd_adjusted.values],columns = R_MA_1ago.index,index = ["R_MA_1ago","R_MA_halfago","MA_now","price",'std','std_f','MA_1ago','MA_halfago','Fall_MDD','R_60','R_60_half','R_fwd',"Rfwd_adjusted"]).T
preselected_stock = preselected_stock.loc[selected_stock_list,:]
preselected_stock["Rfwd_adjusted"]=Rfwd_adjusted.values
preselected_stock["std_f"]=weekly_std.values
preselected_stock["SR"]=preselected_stock["Rfwd_adjusted"]/preselected_stock["std_f"]
preselected_stock["PR_SR"]=preselected_stock["PR"]*preselected_stock["SR"]**0.5
preselected_stock.to_csv("preselected_Stocks_info.csv")
p=FFC.MA_now
EPS_fwd=optimizer.EPS_fwd
EPS_ttm=optimizer.EPS_ttm
fall=FFC.Fall_MDD
FFC.to_csv("WEDataBank.csv")

PE=p/EPS_fwd
PEG= p/(EPS_fwd*(EPS_fwd/EPS_ttm-1))
PEG[PEG<0]=2000


In [186]:
cov = pd.DataFrame(np.cov(np.exp(one_year_weekly_return)-1,rowvar=False,bias = False)*51,index= one_year_weekly_return.columns, columns= one_year_weekly_return.columns)
E_2 = Rfwd_adjusted/fall
E_3 = Rfwd_adjusted/PEG
E_4 = Rfwd_adjusted
std =  np.std(np.exp(one_year_weekly_return)-1)*np.sqrt(52)


# Optimization

In [187]:
w0 = 1/selected_stock_number*np.ones(selected_stock_number)
uplim_ini = np.zeros(selected_stock_number)
uplim3 =pd.DataFrame( w1*(1-uplim_ini) + w2 * uplim_ini,index = std.index,columns=["weight"])
for i in uplim3.index:
    if i in ["SPY","IWD"]:
        uplim3.loc[i,:]=0
low_lim =  np.zeros(selected_stock_number)
multi=lambda x: x.T @cov@x

In [188]:
ef = EfficientFrontier(E_2/100, cov,weight_bounds=(0, w1))
#ef.add_constraint(lambda w: w@E_2== 12)
raw_weights = ef.max_sharpe(0.01)
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose=True)


OrderedDict([('ED', 0.0), ('DUK', 0.17), ('CMS', 0.0), ('ATO', 0.13942), ('CHD', 0.0), ('AEE', 0.0), ('AZO', 0.17), ('EVRG', 0.0), ('AEP', 0.06218), ('AWK', 0.0), ('DTE', 0.0), ('FE', 0.17), ('D', 0.0), ('DLTR', 0.0), ('CNP', 0.1184), ('CLX', 0.0), ('EXC', 0.0), ('CPB', 0.0), ('ES', 0.0), ('AMT', 0.0), ('ETR', 0.17), ('CCI', 0.0)])
Expected annual return: 5.7%
Annual volatility: 14.1%
Sharpe Ratio: 0.33


/Users/l22dwar/anaconda3/lib/python3.11/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  warnings.warn(


(0.05699065898506442, 0.14100671616177618, 0.33325121146110737)

In [189]:
weight1=np.array(pd.DataFrame(cleaned_weights.values(), index = list(cleaned_weights),columns = ["weight"]))[:,0]

In [190]:
sd1 = np.sqrt(weight1@cov@weight1.T)
R1 = weight1.T@E_2/100
sharpe_ratio1 = (R1-0.01)/sd1
print("new_sharpe:",sharpe_ratio1)

new_sharpe: 0.3332512114584867


weight2 =   pd.read_excel("old_weight.xlsx",index_col=0)
weight2=np.array(weight2.loc[list(cleaned_weights),:])[:,0]

sd2 = np.sqrt(weight2@cov@weight2.T)
R2 = weight2.T@E_2/100
sharpe_ratio2 = (R2-0.01)/sd2
print("old_sharpe:",sharpe_ratio2)

In [191]:
weight=pd.DataFrame(cleaned_weights.values(), index = list(cleaned_weights),columns = ["weight"])
weight = weight[weight.weight >=0.030]
weight.weight = weight.weight/sum(weight.weight)
weight.to_csv("weights.csv")

See pyportfolioopt.readthedocs.io/en/latest/MeanVariance.html for more reference


In [192]:
if stock_type == "S&P100":
    OEX=yf.download("^OEX", start=start_swan_date, end=end_swan_date)["Close"]

elif stock_type == "S&P500":
    OEX=yf.download("SPY", start=start_swan_date, end=end_swan_date)["Close"]

stock_price=yf.download(list(weight.index), start=start_swan_date, end=end_swan_date)["Close"]

/var/folders/fm/vxj_bbsj65n9s3s_fqtcvk5r0000gn/T/ipykernel_43704/1453507421.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  OEX=yf.download("SPY", start=start_swan_date, end=end_swan_date)["Close"]
[*********************100%***********************]  1 of 1 completed
/var/folders/fm/vxj_bbsj65n9s3s_fqtcvk5r0000gn/T/ipykernel_43704/1453507421.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_price=yf.download(list(weight.index), start=start_swan_date, end=end_swan_date)["Close"]
[*********************100%***********************]  7 of 7 completed


In [193]:
stock_price.to_csv("tradeinfo.csv")
stock_price=stock_price[weight.index]
stock_price

Ticker,DUK,ATO,AZO,AEP,FE,CNP,ETR
Date,,,,,,,
2022-01-03,89.213409,95.630264,2045.109985,77.036903,35.350346,25.057844,48.401489
2022-01-04,89.144440,96.768623,2080.250000,77.071678,35.213200,24.903614,48.349358
2022-01-05,89.342705,97.078247,2034.030029,77.784576,34.973183,24.849178,48.462322
2022-01-06,89.196152,96.322380,2062.489990,77.532478,35.093189,25.085060,47.641148
2022-01-07,90.084076,95.885262,2014.770020,78.940872,35.496063,25.094135,48.718666
...,...,...,...,...,...,...,...
2025-10-06,124.379997,173.320007,4174.430176,115.660004,46.369999,39.459999,97.480003
2025-10-07,125.559998,174.910004,4085.929932,118.160004,46.410000,39.430000,95.980003
2025-10-08,125.230003,175.750000,4027.760010,118.190002,46.680000,39.560001,96.660004


In [194]:
weight

,weight
DUK,0.17000
ATO,0.13942
AZO,0.17000
AEP,0.06218
FE,0.17000
CNP,0.11840
ETR,0.17000


In [ ]:
def px_on_or_before(series, date_str):
    """series: 1D price series with a DatetimeIndex
       date_str: 'YYYY-MM-DD'
       -> returns the last available closing price on or before the given date"""
    ts = pd.to_datetime(date_str)
    # remove timezone info if present
    if getattr(series.index, "tz", None) is not None:
        series = series.tz_convert(None)
    # safely slice up to the target date
    sub = series.loc[:ts]
    if sub.empty:
        raise ValueError(f"No data on/before {ts.date()} (series starts at {series.index.min().date()})")
    return sub.iloc[-1]


In [ ]:
start = px_on_or_before(stock_price, "2022-02-03")
end   = px_on_or_before(stock_price, "2025-10-13")


oex_start = px_on_or_before(OEX, "2022-02-03")
oex_end   = px_on_or_before(OEX, "2025-10-13")

stock_drop  = abs(end / start - 1)
market_drop = abs(oex_end / oex_start - 1)


In [209]:
stock_drop

Ticker
DUK    0.397578
ATO    0.814850
AZO    1.007374
AEP    0.492656
FE     0.284638
CNP    0.536493
ETR    0.958102
dtype: float64

In [210]:
rest=1
extra=0
for i, row in weight.iterrows():
  if row['weight']>0.17:
    rest=rest-row['weight']
    extra=extra+(row['weight']-0.17)
    # row['weight']=0.17

for i,row in weight.iterrows():
  if row['weight']<0.165:
    # row['weight']=row['weight']+row['weight']/rest*extra
    a=1

weight.to_csv("weights.csv")
weight

,weight
DUK,0.17000
ATO,0.13942
AZO,0.17000
AEP,0.06218
FE,0.17000
CNP,0.11840
ETR,0.17000


In [211]:
portfolio_beta = portfolio_beta.loc[weight.index]
portfolio_beta

,Swan -3,Swan -2,Swan -1,Swan1,Swan2,Swan3,Swan4,Swan5,Swan6,Swan7,Swan8,Swan9,Swan10
DUK,NaN,-0.076048,0.180673,0.422869,-0.127915,-0.518507,1.614607,0.753130,-0.306652,0.261514,-0.370332,1.089892,0.594364
ATO,NaN,NaN,-0.063327,0.081180,-0.126291,0.174381,0.347331,0.548667,0.468100,-0.135667,0.164406,0.959928,0.152146
AZO,NaN,NaN,NaN,NaN,NaN,1.608092,-0.307386,-0.451106,-0.302806,-0.147543,-0.403255,0.955177,-0.400976
AEP,0.716396,-0.363540,-0.067869,0.402325,0.308347,-0.102457,1.807239,0.764573,0.012209,0.305262,-0.296072,0.959958,0.223610
FE,NaN,NaN,NaN,NaN,NaN,-0.142408,0.537585,1.021035,0.025130,0.471489,0.072718,1.078281,0.401794
CNP,0.283846,0.224530,0.180693,0.483945,-0.131417,0.073462,1.992121,0.846555,0.242577,0.267540,-0.032564,1.608039,0.240148
ETR,0.604605,-0.254246,2.248032,0.509609,0.028769,-0.287182,0.241791,0.734601,0.232164,0.918210,-0.246784,1.223640,0.449108


In [212]:
injected_result = pd.DataFrame(index = stock_price.index)
S1 = np.array(10000*weight.T)/np.array(stock_price.iloc[0,:]).T


In [213]:
portfolio_value = np.sum(np.array(weight.T) * stock_price,axis=1)
S1 = np.array(10000*weight.T)/np.array(stock_price.iloc[0,:]).T
injected_result["portfolio_value"]=stock_price@S1.T
S2 = 10000/float(OEX.iloc[0,0])
injected_result["oex"]=S2*OEX
injected_result.to_csv("Injected_result.csv")


In [ ]:
def rows_on_or_before(df, dates):
    """df: DataFrame with a DatetimeIndex (e.g., injected_result)
       dates: list of target dates (strings or timestamps)
       -> returns rows on or before each specified date"""
    out = df.copy()
    out.index = pd.to_datetime(out.index).tz_localize(None)  # remove timezone info
    out = out.sort_index()

    want = pd.to_datetime(pd.Index(dates))
    # align desired dates with existing index, forward-fill from the past
    aligned = out.reindex(out.index.union(want)).ffill()
    return aligned.loc[want]


# Example usage
temp = rows_on_or_before(injected_result, ["2022-02-03", "2025-10-13"])

# compute drawdowns safely with iloc
Portfolio_DD = abs(temp["portfolio_value"].iloc[-1] / temp["portfolio_value"].iloc[0] - 1)
DD           = abs(temp["oex"].iloc[-1]             / temp["oex"].iloc[0]             - 1)

PR = 1 - (Portfolio_DD / DD)
print(PR)
print(temp)


-0.20966630864170832
            portfolio_value           oex
2022-02-03     10141.516369   9348.770935
2025-10-13     16781.450917  14408.763005


In [217]:
allocation=pd.read_excel('allocation weight.xlsx',index_col='stock selected')
dfMVOstock=deepcopy(weight)
dfMVOstock['PR_SR']=''
dfMVOstock['SR']=''
for i in range(len(weight)):
    dfMVOstock.iloc[i,1]=preselected_stock.loc[weight.index[i],'PR_SR']
    dfMVOstock.iloc[i,2]=preselected_stock.loc[weight.index[i],'SR']
dfMVOstock_sort=dfMVOstock.sort_values(by='PR_SR',ascending=False)
postallocationweight=post_allocation(dfMVOstock_sort)
postallocationweight


,weight,PR_SR,SR,equal,proportion,adjusted proportion,direct
AZO,0.17000,1.974672,3.406422,0.142857,0.24766,0.17,0.17
ATO,0.13942,1.221974,4.162899,0.142857,0.153258,0.166201,0.17
DUK,0.17000,1.183516,2.743823,0.142857,0.148434,0.161378,0.17
AEP,0.06218,1.163654,3.196938,0.142857,0.145943,0.158887,0.17
ETR,0.17000,0.920577,4.634213,0.142857,0.115457,0.1284,0.17
FE,0.17000,0.80658,2.99241,0.142857,0.10116,0.114103,0.075
CNP,0.11840,0.702358,2.884152,0.142857,0.088088,0.101032,0.075


In [218]:
#draw type E graph for postallocation
MVOweight=postallocationweight['weight']
equalweight=postallocationweight['equal']
proportionweight=postallocationweight['adjusted proportion']
directweight=postallocationweight['direct']
output(MVOweight).to_csv('no_post_allocation_result.csv')
output(equalweight).to_csv('equal_weight_result.csv')
output(proportionweight).to_csv('proportional_weight_result.csv')
output(directweight).to_csv('direct_weight_result.csv')

/var/folders/fm/vxj_bbsj65n9s3s_fqtcvk5r0000gn/T/ipykernel_43704/3416470394.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  OEX=yf.download("SPY", start=start_swan_date, end=end_swan_date)["Close"]
[*********************100%***********************]  1 of 1 completed
/var/folders/fm/vxj_bbsj65n9s3s_fqtcvk5r0000gn/T/ipykernel_43704/3416470394.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_price=yf.download(list(weight.index), start=start_swan_date, end=end_swan_date)["Close"]
[*********************100%***********************]  7 of 7 completed
/var/folders/fm/vxj_bbsj65n9s3s_fqtcvk5r0000gn/T/ipykernel_43704/3416470394.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  OEX=yf.download("SPY", start=start_swan_date, end=end_swan_date)["Close"]
[*********************100%***********************]  1 of 1 completed
/var/folders/fm/vxj_bbsj65n9s3s_fqtcvk5r0000gn/T/ipykernel_43704

#